In [1]:
%matplotlib inline
from PIL import Image
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import os
import itertools
from tqdm import *
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import math
import matplotlib.pyplot as plt

from IPython.display import clear_output

from io import StringIO
import re

import pickle
import os

from multiprocessing import Pool 
from itertools import product
import sys
from functools import partial

# Step 1: Load Data Set

In [2]:
fileNames = []
for file in os.listdir("data/train_1"):
    if file.endswith(".jpg"):
        fileNames.append(file)
# fileNamesDF = pd.DataFrame(fileNames)

In [3]:
#only take certain artist so that the probabilities aren't basically all false when doing the tupples
minNumPaintingsPerArtist = 55

# #for each artist, only take n of their paintings
numPaintingsPerAuthor = 100


trainInfo = pd.read_csv('data/train_info.csv')
trainInfo = trainInfo[trainInfo['filename'].isin(fileNames)]

mostPopularArtists = trainInfo['artist'].value_counts()[trainInfo['artist'].value_counts() > minNumPaintingsPerArtist]
# print(mostPopularArtists.head())
# fileNamesDFAll = trainInfo[trainInfo['artist'].isin(mostPopularArtists.index)]
# print(fileNamesDFAll.head())

fileNamesDF = pd.DataFrame()
moreInfo = pd.DataFrame()
for artist in tqdm(mostPopularArtists.index):
    fileNamesDF = fileNamesDF.append(trainInfo[trainInfo['artist'] == artist][:numPaintingsPerAuthor])
    
fileNamesDF.set_index('filename', inplace=True)

In [4]:
# Load the all_data_info for the dataset we are using
allInfo = pd.read_csv('data/all_data_info.csv')
allInfo.set_index('new_filename', inplace=True)

allInfo = allInfo[allInfo.index.isin(fileNamesDF.index)]


featuresDF = pd.DataFrame(index=allInfo.index)
featuresDF['pixelsx'] = allInfo['pixelsx']
featuresDF['pixelsy'] = allInfo['pixelsy']
featuresDF['style'] = allInfo['style']
featuresDF['genre'] = fileNamesDF['genre']


# featuresDF['features'] = pd.Series()

In [5]:
allInfo

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train
new_filename,,,,,,,,,,,
15075.jpg,Albrecht Durer,NaN,sketch and study,2536.0,2929.0,621267.0,wikiart,Northern Renaissance,Study of a body of Christ,train_and_test,True
18039.jpg,Albrecht Durer,1506,religious painting,2352.0,3129.0,1046324.0,wikiart,Northern Renaissance,Madonna with the Siskin,train_and_test,True
1053.jpg,Albrecht Durer,NaN,design,2536.0,2836.0,725632.0,wikiart,Northern Renaissance,Wappen Pirckheimer,train_and_test,True
1458.jpg,Albrecht Durer,NaN,miniature,2394.0,3000.0,693310.0,wikiart,Northern Renaissance,Draft for a book sign lion,train_and_test,True
18076.jpg,Odilon Redon,NaN,flower painting,2232.0,3198.0,3950198.0,wikiart,Realism,The Red Poppy,train_and_test,True
13849.jpg,Albrecht Durer,NaN,portrait,2360.0,3000.0,757619.0,wikiart,Northern Renaissance,Portrait of a Woman,train_and_test,True
1805.jpg,Albrecht Durer,NaN,portrait,2220.0,3000.0,830408.0,wikiart,Northern Renaissance,Portrait of Conrat Verkell,train_and_test,True
19349.jpg,Odilon Redon,1900,symbolic painting,2536.0,2602.0,497502.0,wikiart,Symbolism,Silence,train_and_test,True
10199.jpg,Albrecht Durer,NaN,religious painting,2187.0,3000.0,850580.0,wikipedia,Northern Renaissance,Christ on the Mount of Olives,train_and_test,True


In [6]:
def normalizeSeries(series, maxNum):
    series = series/maxNum
    return series

In [88]:
def featurizeImage(filename):
    # Extract the features from the actual image
    jpgfile = Image.open("data/train_1/" + filename)
#     print(jpgfile)
    
    bands = jpgfile.getbands()
    if bands[0] == 'L':
        # grayscale image
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['grayscale'])
        imageDF['red'] = 0
        imageDF['green'] = 0
        imageDF['blue'] = 0
        imageDF['alpha'] = 0
#         return imageDF
#         imageDF['grayscale'] = normalizeSeries(imageDF['grayscale',])
    elif len(bands) == 4:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue','alpha'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
        imageDF['alpha'] = normalizeSeries(imageDF['alpha'],255) 
        imageDF['grayscale'] = 0 
    elif len(bands) == 3:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
        imageDF['alpha'] = 0  
        imageDF['grayscale'] = 0 
    else:
        imageDF = pd.DataFrame(list(jpgfile.getdata()))
        print(bands)
        return imageDF.mean(), bands
    
    
    
    return imageDF.mean()

In [15]:
def pickleDF(fileName,dataFrame,numSplits):
    print(numSplits)
    dfArray = np.array_split(dataFrame, numSplits)
    for i, df in enumerate(dfArray):
        df.to_pickle(fileName+"-"+str(i))

In [97]:
def getImgData(dataFrame,baseName):
    sys.stdout.flush()
#     index = 0
#     while os.path.exists(baseName+"-"+str(index)):
#         index = index + 1
    
    newFileName = baseName + ".pckl"
    print(newFileName)
    sys.stdout.flush()
#     featureSeries = []
    reds = []
    greens = []
    blues = []
    alphas = []
    grayscales = []
    for row in tqdm(list(dataFrame.index), leave=False): 
        imageFeatures = featurizeImage(row)
        reds.append(imageFeatures[0])
        greens.append(imageFeatures[1])
        blues.append(imageFeatures[2])
        alphas.append(imageFeatures[3])
        grayscales.append(imageFeatures[4])

        sys.stdout.flush()
    
#     print(featureSeries)
    dataFrame['red'] = reds
    dataFrame['green'] = greens
    dataFrame['blue'] = blues
    dataFrame['alpha'] = alphas
    dataFrame['grayscale'] = grayscales


#     print(dataFrame['features'])
    dataFrame.to_pickle(newFileName)
    return dataFrame

In [98]:
def concatFrames(dataFrames):
    return pd.concat(dataFrames)

In [99]:
def loadImagesFromPickles(fileNames):
    
    dataFrames = []
    for fileName in fileNames:
        dataFrames.append(pd.read_pickle(fileName))

    return concatFrames(dataFrames)

In [100]:
baseFileName = 'trainImgsAllcolorBands-NewFeaturesZeroes'
splitFeaturesData = np.array_split(featuresDF,10)
print(len(splitFeaturesData))
pool = Pool()
inputData = [[splitFeaturesData[i], baseFileName+"-"+str(i)] for i in range(len(splitFeaturesData))]
pool.starmap(getImgData, inputData)
pool.close() 
results = pool.join()

10
trainImgsAllcolorBands-NewFeaturesZeroes-1.pckl
trainImgsAllcolorBands-NewFeaturesZeroes-0.pckl
trainImgsAllcolorBands-NewFeaturesZeroes-3.pckl
trainImgsAllcolorBands-NewFeaturesZeroes-2.pckl


trainImgsAllcolorBands-NewFeaturesZeroes-4.pckl


trainImgsAllcolorBands-NewFeaturesZeroes-5.pckl


trainImgsAllcolorBands-NewFeaturesZeroes-6.pckl


trainImgsAllcolorBands-NewFeaturesZeroes-7.pckl


trainImgsAllcolorBands-NewFeaturesZeroes-8.pckl


trainImgsAllcolorBands-NewFeaturesZeroes-9.pckl


In [101]:
print(results)

None


# Models

In [132]:
pickleLoaded = loadImagesFromPickles([baseFileName+"-"+str(i)+".pckl" for i in range(len(splitFeaturesData))])

In [146]:
uniqueLabels = pd.DataFrame(allInfo.loc[pickleLoaded.index]['artist'].unique(),columns=['artist'])
uniqueStyles = pd.DataFrame(pickleLoaded['style'].unique(),columns=['style'])
uniqueGenre = pd.DataFrame(pickleLoaded['genre'].unique(),columns=['genre'])

In [147]:
pickleLoaded['style'] = pickleLoaded['style'].apply(lambda x: uniqueStyles[uniqueStyles['style'] == x].index.values[0])
pickleLoaded['genre'] = pickleLoaded['genre'].apply(lambda x: uniqueGenre[uniqueGenre['genre'] == x].index.values[0])

In [149]:
nTrainFraction = 0.6
featuresTrain = pickleLoaded.sample(frac=nTrainFraction)
featuresDev = pickleLoaded.drop(featuresTrain.index)

In [156]:
labelsTrain = allLabels.loc[featuresTrain.index]
labelsTrainValue = labelsTrain.apply(lambda x: uniqueLabels[uniqueLabels['artist'] == x].index.values[0])

labelsDev = allLabels.loc[featuresDev.index]
labelsDevValue = labelsDev.apply(lambda x: uniqueLabels[uniqueLabels['artist'] == x].index.values[0])

In [160]:
classifier = RandomForestClassifier()
classifier.fit(featuresTrain,labelsTrainValue)
predictions = classifier.predict(featuresDev)

In [161]:
labelsDevValue == predictions

new_filename
15075.jpg      True
1053.jpg       True
1458.jpg       True
1805.jpg       True
10199.jpg      True
12105.jpg      True
103162.jpg     True
13513.jpg      True
19114.jpg      True
19728.jpg      True
1975.jpg       True
101535.jpg     True
11430.jpg      True
19187.jpg      True
19584.jpg      True
18483.jpg      True
15551.jpg      True
12966.jpg      True
19310.jpg      True
17656.jpg     False
11851.jpg      True
12660.jpg      True
102050.jpg     True
10135.jpg      True
19830.jpg      True
18807.jpg      True
101029.jpg    False
11466.jpg      True
12390.jpg     False
101608.jpg     True
              ...  
101956.jpg     True
16420.jpg      True
17678.jpg      True
17929.jpg      True
16796.jpg     False
12988.jpg      True
12148.jpg      True
101696.jpg    False
101144.jpg     True
12675.jpg      True
102911.jpg    False
15101.jpg      True
103033.jpg     True
102760.jpg     True
19597.jpg      True
17866.jpg     False
10185.jpg      True
15737.jpg      True
13982.j